In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from IPython.display import display

%config InlineBackend.figure_format = 'retina'

sns.set_theme()

In [13]:
data = pd.DataFrame()

# "logs/float16-run2/", "logs/float32-run2/"]:
for log_path in ["logs/float16-run1/", "logs/float32-run1/", ]:
    df = pd.read_csv(log_path + "data.csv", index_col=0, )
    df["type"] = log_path.split("/")[-2]
    data = pd.concat([data, df])

sns.relplot(
    data=data,
    kind="line",
    x="time", y="p_loss",
    hue="type",
)

# set fig size

plt.yscale("log")
plt.show()

ValueError: Length of tuple vectors must match length of `data` when both are used, but `data` has length 948 and the vector passed to `size` has length 2.